In [8]:
!pip install google-cloud-translate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 2.4 MB/s eta 0:00:00a 0:00:01


In [5]:
!pip install google-cloud-language

In [1]:
# General
import pandas as pd

# Main
from vertexai import evaluation
from vertexai.evaluation.metrics import pointwise_metric

In [2]:
# @title Helper functions
from IPython.display import Markdown, display


def display_eval_result(eval_result, metrics=None, model_name=None, rows=0):
    if model_name is not None:
        display(Markdown("## Eval Result for %s" % model_name))

    """Display the evaluation results."""
    summary_metrics, metrics_table = (
        eval_result.summary_metrics,
        eval_result.metrics_table,
    )

    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        metrics_table = metrics_table.filter(
            [
                metric
                for metric in metrics_table.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the summary metrics
    display(Markdown("### Summary Metrics"))
    display(metrics_df)
    if rows > 0:
        # Display samples from the metrics table
        display(Markdown("### Row-based Metrics"))
        display(metrics_table.head(rows))

# Set up eval metrics for your data.

You can evaluate the translation quality of your data generated from an LLM using:
- [BLEU](https://en.wikipedia.org/wiki/BLEU)
- [COMET](https://unbabel.github.io/COMET/html/index.html)
- [MetricX](https://github.com/google-research/metricx)

In [3]:
metrics = [
    "bleu",
    pointwise_metric.Comet(),
    pointwise_metric.MetricX(),
]

In [31]:
PROJECT_ID='nine-quality-test'
# Imports the Google Cloud Translation library
import os

from google.cloud import translate_v3


# Initialize Translation client
def translate_text(
    text: str = "YOUR_TEXT_TO_TRANSLATE",
    language_code: str = "fr",
    source_language: str="en-US"
) -> translate_v3.TranslationServiceClient:
    """Translating Text from English.
    Args:
        text: The content to translate.
        language_code: The language code for the translation.
            E.g. "fr" for French, "es" for Spanish, etc.
            Available languages: https://cloud.google.com/translate/docs/languages#neural_machine_translation_model
    """

    client = translate_v3.TranslationServiceClient()
    parent = f"projects/{PROJECT_ID}/locations/global"
    # Translate text from English to chosen language
    # Supported mime types: # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        contents=[text],
        target_language_code=language_code,
        parent=parent,
        mime_type="text/plain",
        source_language_code=source_language,
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print(f"Translated text: {translation.translated_text}")
    # Example response:
    # Translated text: Bonjour comment vas-tu aujourd'hui?

    return response


In [32]:
# Text to translate
input_text = "Dem Feuer konnte Einhalt geboten werden"
source_language="de"    
# Target language code (e.g., 'es' for Spanish)
target_language = "en-US"
    
# Call the translation function
translated_text = translate_text(input_text, target_language,source_language)
    
print(f"Original: {input_text}")
print(f"Translated: {translated_text}")

Translated text: The fire was stopped
Original: Dem Feuer konnte Einhalt geboten werden
Translated: translations {
  translated_text: "The fire was stopped"
}



In [33]:
# Text to translate
input_text = "Schulen und Kindergärten wurden eröffnet."
source_language="de"    
# Target language code (e.g., 'es' for Spanish)
target_language = "en-US"
    
# Call the translation function
translated_text = translate_text(input_text, target_language,source_language)
    
print(f"Original: {input_text}")
print(f"Translated: {translated_text}")

Translated text: Schools and kindergartens were opened.
Original: Schulen und Kindergärten wurden eröffnet.
Translated: translations {
  translated_text: "Schools and kindergartens were opened."
}



# Prepare your dataset

Evaluate stored generative AI model responses in an evaluation dataset.

In [34]:
sources = [
    "Dem Feuer konnte Einhalt geboten werden",
    "Schulen und Kindergärten wurden eröffnet.",
]

responses = [
    "The fire could be stopped",
    "Schools and kindergartens were open",
]

references = [
    "The fire was stopped",
    "Schools and kindergartens were opened.",
]

eval_dataset = pd.DataFrame(
    {
        "source": sources,
        "response": responses,
        "reference": references,
    }
)

# Run evaluation

With the evaluation dataset and metrics defined, you can run evaluation for an `EvalTask` on different models and applications, and many other use cases.

In [35]:

eval_task = evaluation.EvalTask(
    dataset=eval_dataset, metrics=metrics, experiment=EXPERIMENT_NAME
)
eval_result = eval_task.evaluate()

NameError: name 'EXPERIMENT_NAME' is not defined

You can view the summary metrics and row-based metrics for each response in the `EvalResult`.


In [ ]:
display_eval_result(eval_result, rows=2)

# Clean up

Delete ExperimentRun created by the evaluation.

In [ ]:
from google.cloud import aiplatform

aiplatform.ExperimentRun(
    run_name=eval_result.metadata["experiment_run"],
    experiment=eval_result.metadata["experiment"],
).delete()